# Flu shot acceptance: learning from a previous pandemic

This is the capstone project of the neue fische Data Science Bootcamp. It was a collaborative effort between [Christina Rudolf](https://github.com/christinarudolf), [Raymond Boateng](https://github.com/RayKwame) and [Juliane Berek](https://github.com/julianeberek).

## Introduction

The H1N1 influenza pandemic (also known as swine influenza) was caused by H1N1 influenza virus and affected the world between June 2009 to August 2010 (according to WHO declaration). It was the most recent pandemic prior to COVID19. An estimated 11 - 21% of the global population was affected, with deaths in the U.S. totalling 12,469.

This project aimed to:
- Help to increase vaccination rates for seasonal and pandemic flu in the overall population (this reducing the burden of influenza by decreasing hospitalisations/ deaths)
- Identify factors that determine the chance of getting vaccinated
- Identify groups with lower likelihood for getting vaccinated, in order to target them with promotions
- Determine differences of seasonal vs. H1N1 (pandemic) vaccinations

## Description of dataset

The original data was collected through the National 2009 H1N1 Flu Survey in the U.S. between 2009 - 2010. The current dataset formed part of the DrivenData Challenge ["Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines"](https://www.drivendata.org/competitions/66/flu-shot-learning/).

The dataset consists of approximately 27 000 participant responses. The main outcome variables in the dataset:
- whether the participant received the vaccination against the H1N1 flu
- whether the participant received the vaccination against the seasonal flu

The remainder of the dataset consists of [35 categorical variables](https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/), broadly falling into participant demographics, attitudes and knowledge about H1N1 and seasonal flu and vaccination, and healthcare information.

## Hypotheses

We had two main hypotheses we intended to address in our project:

- Some features affect the likelihood of vaccination more than others, e.g. attitudes and knowledge, recommendations by doctors
- H1N1 vaccination is taken more due to the pandemic context 

## Notebook description

![](https://raw.githubusercontent.com/RayKwame/TheFluShot/main/images/Model_creation.JPG?token=ATDLUYIYB7MTMIFTWAQWZS3BCFWO6)
This notebook describes the steps taken to create the final prediction models for predicting vaccination against the flu (for the H1N1 and seasonal flu respectively). It also describes the identification and exploration of the important features for vaccination prediction.

It includes the following sections:

1. Import of necessary packages and libraries
2. Import and examination of data
3. Data cleaning
4. EDA
5. Feature engineering
6. Prediction of H1N1 flu vaccination
    6.1 Data balancing by downsampling the majority class
    6.2 Modelling and prediction
    6.3 Evaluation of model performance
    6.4 Hyperparameter tuning
    6.5 Error evaluation of model
7. Prediction of seasonal flu vaccination
    7.1 Data balancing by downsampling the majority class
    7.2 Modelling and prediction
    7.3 Evaluation of model performance
    7.4 Hyperparameter tuning
    7.5 Error evaluation of model
8. Summary and conclusions

In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
#from modeling.config import TRACKING_URI, EXPERIMENT_NAME

RSEED = 42
# Modeling Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

# for resampling (data balancing)
from sklearn.utils import resample

# for feature importance
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

warnings.filterwarnings('ignore')

     |████████████████████████████████| 106 kB 3.6 MB/s eta 0:00:01


In [3]:
df = pd.read_csv('data/Flu_Shot_Data_cleaned_2.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   26707 non-null  int64  
 1   h1n1_vaccine                 26707 non-null  int64  
 2   seasonal_vaccine             26707 non-null  int64  
 3   h1n1_concern                 26615 non-null  float64
 4   h1n1_knowledge               26591 non-null  float64
 5   behavioral_antiviral_meds    26636 non-null  float64
 6   behavioral_avoidance         26499 non-null  float64
 7   behavioral_face_mask         26688 non-null  float64
 8   behavioral_wash_hands        26665 non-null  float64
 9   behavioral_large_gatherings  26620 non-null  float64
 10  behavioral_outside_home      26625 non-null  float64
 11  behavioral_touch_face        26579 non-null  float64
 12  doctor_recc_h1n1             24547 non-null  float64
 13  doctor_recc_seas